# Continuum Quickstart

**Decisions that stick.** Continuum is a decision management framework for AI-assisted coding.
It captures, enforces, and recalls decisions so your team (and your agents) never re-debate the same thing twice.

This notebook walks through the core workflow in under 5 minutes:

1. **Commit** a decision
2. **Inspect** the active binding set
3. **Enforce** an action against decisions
4. **Resolve** an ambiguous prompt (ambiguity gate)
5. **Sticky resolution** — the same prompt resolves instantly the second time
6. **Supersede** a decision with a newer version

No API keys, no servers — everything runs locally in this notebook.

---

## Install the Continuum SDK

One package, zero configuration.

In [ ]:
%pip install -q continuum-sdk

## Setup

Create a `ContinuumClient` pointed at a clean temporary directory.
All decisions are stored as local JSON files — no database needed.

In [ ]:
import json
import shutil
from pathlib import Path
from continuum.client import ContinuumClient

# Start with a clean store each time you run this notebook
STORE_DIR = Path("/tmp/continuum-quickstart")
if STORE_DIR.exists():
    shutil.rmtree(STORE_DIR)

client = ContinuumClient(storage_dir=str(STORE_DIR))
SCOPE = "repo:continuum-demo"

print(f"Continuum client ready.")
print(f"Storage: {STORE_DIR}")
print(f"Scope:   {SCOPE}")

---
## Step 1: Commit a Decision

A **rejection decision** declares that a particular approach is off-limits.
Here we reject "full rewrites" in favour of incremental refactors.

Once committed and activated, this decision becomes part of the **binding set**
for the scope — any agent or tool that checks Continuum will know about it.

In [ ]:
reject_dec = client.commit(
    title="Reject full rewrites in this repo",
    scope=SCOPE,
    decision_type="rejection",
    options=[
        {"id": "opt_incremental", "title": "Incremental refactor", "selected": True},
        {
            "id": "opt_full_rewrite",
            "title": "Full rewrite",
            "selected": False,
            "rejected_reason": "Too risky; prefer incremental refactors.",
        },
    ],
    rationale="Prefer incremental refactors to reduce risk and review cost.",
)

# Activate the decision
reject_dec = client.update_status(reject_dec.id, "active")

print(f"Decision committed and activated!")
print(f"  ID:     {reject_dec.id}")
print(f"  Title:  {reject_dec.title}")
print(f"  Status: {reject_dec.status}")

---
## Step 2: Inspect the Binding Set

The **binding set** is the list of active decisions for a given scope.
Think of it as the "constitution" that agents must follow when working in this context.

In [ ]:
binding = client.inspect(SCOPE)

print(f"Active decisions in scope '{SCOPE}': {len(binding)}\n")
print(json.dumps(binding, indent=2, default=str))

---
## Step 3: Enforce an Action

The **enforcement engine** evaluates a proposed action against the binding set.

- A "full rewrite" should be **blocked** (violates the rejection decision).
- Adding error handling should be **allowed** (no conflict).

This is how Continuum prevents agents from repeating mistakes or violating team decisions.

In [ ]:
# This should be BLOCKED — violates the rejection
banned_action = {
    "type": "code_change",
    "description": "Do a full rewrite of auth module",
}
result = client.enforce(banned_action, SCOPE)
print(f"Action: '{banned_action['description']}'")
print(f"  Verdict: {result['verdict'].upper()}")
print(f"  Reason:  {result['reason']}")

print()

# This should be ALLOWED — no conflict
safe_action = {
    "type": "code_change",
    "description": "Add error handling to auth module",
}
safe_result = client.enforce(safe_action, SCOPE)
print(f"Action: '{safe_action['description']}'")
print(f"  Verdict: {safe_result['verdict'].upper()}")
print(f"  Reason:  {safe_result['reason']}")

---
## Step 4: Ambiguity Gate

When a prompt is **ambiguous**, Continuum flags it and asks for clarification
instead of letting the agent guess.

"Make this production-ready" could mean many things. The **resolve** call detects
that no prior decision covers this phrase and returns `needs_clarification` with
candidate interpretations.

In [ ]:
query = "Make this production-ready"
candidates = [
    {"id": "opt_tests_errors", "title": "Add tests + error handling (repo standard)"},
    {"id": "opt_enterprise", "title": "Enterprise hardening (observability, SLOs, etc.)"},
]

res1 = client.resolve(query=query, scope=SCOPE, candidates=candidates)

print(f"Prompt:  '{query}'")
print(f"Status:  {res1['status']}")
print()
if res1['status'] == 'needs_clarification':
    print("Clarification needed:")
    print(json.dumps(res1.get('clarification'), indent=2))

---
## Step 5: Commit an Interpretation

The user (or agent) picks an option and commits it as an **interpretation decision**.
From now on, "production-ready" in this scope means "tests + error handling".

This is the key insight: **clarify once, remember forever.**

In [ ]:
interp_dec = client.commit(
    title="production-ready",
    scope=SCOPE,
    decision_type="interpretation",
    rationale="In this repo, production-ready means tests + error handling.",
    metadata={"selected_option_id": "opt_tests_errors"},
)
interp_dec = client.update_status(interp_dec.id, "active")

print(f"Interpretation committed!")
print(f"  ID:        {interp_dec.id}")
print(f"  Title:     {interp_dec.title}")
print(f"  Rationale: {interp_dec.rationale}")

---
## Step 6: Sticky Resolution

Now resolve the same prompt again. This time, Continuum finds the committed
interpretation and returns `resolved` — no clarification gate, no re-debate.

In [ ]:
res2 = client.resolve(query=query, scope=SCOPE, candidates=candidates)

print(f"Prompt:  '{query}'")
print(f"Status:  {res2['status']}")
if res2['status'] == 'resolved':
    print(f"Matched: {res2.get('matched_decision_id')}")
    print()
    print("The ambiguity gate is gone — the decision sticks.")

---
## Step 7: Supersede a Decision

Decisions evolve. **Supersession** replaces an old decision with a new one.
The old decision is marked `superseded` and the new one becomes `active`.

The binding set updates deterministically — no stale decisions linger.

In [ ]:
new_dec = client.supersede(
    old_id=interp_dec.id,
    new_title="production-ready",
    rationale="Production-ready now includes tests + error handling + lint.",
    metadata={"selected_option_id": "opt_tests_errors_lint"},
)

print(f"Superseded!")
print(f"  Old: {interp_dec.id} -> status: {client.get(interp_dec.id).status}")
print(f"  New: {new_dec.id} -> status: {new_dec.status}")
print()

# Inspect the updated binding set
binding2 = client.inspect(SCOPE)
print(f"Active decisions in scope '{SCOPE}': {len(binding2)}")
for d in binding2:
    print(f"  - {d['title']} (id={d['id']}, status={d['status']})")

---
## Summary

You just walked through the complete Continuum lifecycle:

| Step | What happened |
|------|---------------|
| **Commit** | Created a rejection decision — "no full rewrites" |
| **Inspect** | Viewed the active binding set for the scope |
| **Enforce** | Blocked a banned action, allowed a safe one |
| **Resolve** | Detected ambiguity in "production-ready" |
| **Interpret** | Committed a sticky interpretation |
| **Resolve again** | Same prompt resolved instantly — no re-debate |
| **Supersede** | Evolved the interpretation to a newer version |

### Next steps

- **Install the CLI:** `pip install continuum-cli` — manage decisions from the terminal
- **MCP Server:** `pip install continuum-mcp-server` — expose decisions to AI agents in Cursor/Claude
- **Docs:** [https://docs.getcontinuum.ai](https://docs.getcontinuum.ai)
- **GitHub:** [https://github.com/get-continuum/continuum](https://github.com/get-continuum/continuum)